In [1]:
import os
import modal
import hopsworks
import pandas as pd
import csv
import json
import os
from sklearn.preprocessing import OneHotEncoder
from pandas.core.frame import DataFrame
from sklearn.model_selection import train_test_split
import numpy as np

## Feature engineer

In [2]:
weathers = pd.read_csv('./Weather/kiruna 2021-01-01 to 2021-12-31.csv')
weathers = weathers[12:8760]
weathers = weathers.reset_index(drop=True)

In [3]:
weather_icon = weathers['icon'] # get weather icons
print(set(weather_icon))
weather_icon = DataFrame(weather_icon) # turn the data type into DataFrame
weather_conditions = weathers['conditions'] # get weather conditions
print(set(weather_conditions))
weather_conditions = DataFrame(weather_conditions) # turn the data type into DataFrame
weather_cloudcover = weathers['cloudcover'] # get weather cloudcover
weather_cloudcover = DataFrame(weather_cloudcover) # turn the data type into DataFrame
weather_visibility = weathers['visibility'] # get weather visibility
weather_visibility = DataFrame(weather_visibility) # turn the data type into DataFrame

{'cloudy', 'snow', 'partly-cloudy-night', 'clear-day', 'clear-night', 'fog', 'rain', 'wind', 'partly-cloudy-day'}
{'Overcast', 'Snow, Overcast', 'Rain, Overcast', 'Rain, Partially cloudy', 'Snow, Rain, Partially cloudy', 'Snow, Partially cloudy', 'Snow, Rain', 'Snow, Rain, Overcast', 'Snow', 'Partially cloudy', 'Clear', 'Rain'}


weather_icon = weathers['icon'] # get weather icons
print(set(weather_icon))
weather_icon = DataFrame(weather_icon) # turn the data type into DataFrame
weather_conditions = weathers['conditions'] # get weather conditions
print(set(weather_conditions))
weather_conditions = DataFrame(weather_conditions) # turn the data type into DataFrame
weather_cloudcover = weathers['cloudcover'] # get weather cloudcover
weather_cloudcover = DataFrame(weather_cloudcover) # turn the data type into DataFrame
weather_visibility = weathers['visibility'] # get weather visibility
weather_visibility = DataFrame(weather_visibility) # turn the data type into DataFrame

In [4]:
encoder = OneHotEncoder(handle_unknown='ignore')
weather_icon_onehot = pd.get_dummies(weather_icon['icon'])
weather_conditions_onehot = pd.get_dummies(weather_conditions['conditions'])

In [5]:
weather_icon_onehot

,clear-day,clear-night,cloudy,fog,partly-cloudy-day,partly-cloudy-night,rain,snow,wind
0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
8743,1,0,0,0,0,0,0,0,0
8744,1,0,0,0,0,0,0,0,0
8745,1,0,0,0,0,0,0,0,0
8746,1,0,0,0,0,0,0,0,0


In [6]:
weather_conditions_onehot

,Clear,Overcast,Partially cloudy,Rain,"Rain, Overcast","Rain, Partially cloudy",Snow,"Snow, Overcast","Snow, Partially cloudy","Snow, Rain","Snow, Rain, Overcast","Snow, Rain, Partially cloudy"
0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8743,1,0,0,0,0,0,0,0,0,0,0,0
8744,1,0,0,0,0,0,0,0,0,0,0,0
8745,1,0,0,0,0,0,0,0,0,0,0,0
8746,1,0,0,0,0,0,0,0,0,0,0,0


In [7]:
with open('./Kp_index/kp.gfz-potsdam.de.json') as f:
    df_Kp = json.load(f)
Kp_index = df_Kp['Kp']
Kp_index = Kp_index[4:2920]
Kp_index.append(2.333) # add the Kp-index of 2021-12-31-24 (2022-01-01-0)
Kp = []
for i in range(0,len(Kp_index)):
    Kp.append(Kp_index[i])
    Kp.append(Kp_index[i])
    Kp.append(Kp_index[i])
Kp = Kp[1:len(Kp)-2]
Kp = DataFrame(Kp)
Kp = Kp.rename(columns={0:'Kp_index'})
Kp

,Kp_index
0,0.333
1,0.333
2,1.333
3,1.333
4,1.333
...,...
8743,0.667
8744,1.000
8745,1.000
8746,1.000


In [8]:
with open('./Aurora_pic/Aurora.json') as f:
    df_Labels = json.load(f)
Labels = df_Labels
Labels = Labels[0:8748]
Labels = DataFrame(Labels)
Labels = Labels.rename(columns={0:'Aurora_Label'})
Labels

,Aurora_Label
0,0
1,0
2,0
3,0
4,0
...,...
8743,0
8744,0
8745,0
8746,0


In [9]:
df_Features = pd.concat([Labels, Kp, weather_cloudcover, weather_visibility, weather_icon_onehot], axis=1)
df_Features = df_Features.drop(index=[1206, 3429]) # drop nan
df_Features = df_Features.reset_index(drop=True)
df_Features.to_csv('df_Features.csv',index= False)
Labels = Labels.drop(index=[1206, 3429])
Labels = Labels.reset_index(drop=True)